In [2]:
import os
import json
from pathlib import Path
from typing import List, Dict

import fitz  # PyMuPDF
from dotenv import load_dotenv

from openai import AsyncOpenAI, RateLimitError
import google.generativeai as genai

# ─────────────────────────────────────────────
# 1️⃣ Load API keys from .env
# ─────────────────────────────────────────────
load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_KEY = os.getenv("GEMINI_API_KEY")

if not OPENAI_KEY and not GEMINI_KEY:
    raise ValueError("❌ No API key configured. Set OPENAI_API_KEY and/or GEMINI_API_KEY in .env")

openai_client = AsyncOpenAI(api_key=OPENAI_KEY) if OPENAI_KEY else None

gemini_model = None
if GEMINI_KEY:
    genai.configure(api_key=GEMINI_KEY)
    gemini_model = genai.GenerativeModel("gemini-1.5-flash")  # valid Gemini model name

# ─────────────────────────────────────────────
# 2️⃣ Resume folder (PDFs)
# ─────────────────────────────────────────────
CURRENT_DIR = Path.cwd()
RESUME_DIR = CURRENT_DIR / "resume"   # e.g. Resume Keyword Search Agent/resume

print(f"📂 Using resume folder: {RESUME_DIR}")
if not RESUME_DIR.exists():
    print("⚠️ Warning: resume folder does not exist, but code will still run.")

# ─────────────────────────────────────────────
# 3️⃣ STEP 1 – Extract keywords from job description
# ─────────────────────────────────────────────
async def extract_keywords_from_job_description(
    job_text: str,
    n_keywords: int = 15,
) -> List[str]:
    """
    Use OpenAI (or Gemini as fallback) to extract the most
    important skills/technologies from a job description.
    Returns a list of keyword strings.
    """

    # 3.1 – Try OpenAI first (chat.completions)
    if openai_client:
        system_msg = (
            "You are a job recruiter. Extract the most important skills, "
            "technologies, and keywords from the job description. "
            f"Return at most {n_keywords} items as a comma-separated list "
            "or a JSON array of strings."
        )
        try:
            completion = await openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": job_text},
                ],
                temperature=0.3,
            )
            raw = completion.choices[0].message.content.strip()
            # Try JSON first
            try:
                if raw.startswith("["):
                    kw_list = json.loads(raw)
                else:
                    raise ValueError
            except Exception:
                # Fallback: split by comma
                kw_list = [k.strip(" .-") for k in raw.split(",") if k.strip()]

            return kw_list[:n_keywords]

        except RateLimitError:
            print("⚠️ OpenAI quota exceeded while extracting keywords. Falling back to Gemini…")
        except Exception as e:
            print(f"⚠️ OpenAI keyword extraction failed ({e}). Trying Gemini if available…")

    # 3.2 – Fallback: Gemini keyword extraction
    if gemini_model:
        try:
            prompt = (
                "You are a job recruiter. From the following job description, "
                f"extract the top {n_keywords} most important skills, technologies, "
                "and keywords. Return ONLY a JSON array of strings, no explanation.\n\n"
                f"Job description:\n{job_text}"
            )
            resp = gemini_model.generate_content(prompt)
            txt = resp.text.strip()

            try:
                kw_list = json.loads(txt)
            except Exception:
                kw_list = [k.strip(" .-") for k in txt.split(",") if k.strip()]

            return kw_list[:n_keywords]
        except Exception as e:
            print(f"⚠️ Gemini keyword extraction failed: {e}")

    # 3.3 – Last fallback: naive
    print("⚠️ All keyword extraction backends failed. Using naive split.")
    words = [w.strip(".,;:()[]") for w in job_text.split()]
    return list(dict.fromkeys(words))[:n_keywords]  # unique order-preserving


# ─────────────────────────────────────────────
# 4️⃣ STEP 2 – Scan PDF resumes for keywords
# ─────────────────────────────────────────────
def scan_resumes_for_keywords(keywords: List[str]) -> List[Dict]:
    """
    Read all PDFs in RESUME_DIR and count how many of the given keywords
    appear in each resume (case-insensitive).
    """
    results: List[Dict] = []

    if not RESUME_DIR.exists():
        return [{"error": f"Resume folder not found: {RESUME_DIR}"}]

    for file in RESUME_DIR.glob("*.pdf"):
        try:
            doc = fitz.open(file)
            text = "\n".join(page.get_text("text") for page in doc)
            doc.close()
        except Exception as e:
            results.append({
                "filename": file.name,
                "match_count": 0,
                "matching_keywords": [],
                "error": f"Failed to read PDF: {e}",
            })
            continue

        lowered = text.lower()
        matched = [kw for kw in keywords if kw.lower() in lowered]

        results.append({
            "filename": file.name,
            "match_count": len(matched),
            "matching_keywords": matched,
        })

    return results


# ─────────────────────────────────────────────
# 5️⃣ STEP 3 – High-level analysis with OpenAI / Gemini
# ─────────────────────────────────────────────
async def analyze_with_openai(required_skills: List[str], scan_results: List[Dict]) -> str:
    """
    Use OpenAI (Responses API) to analyze PDF scan results.
    """
    if not openai_client:
        raise RuntimeError("OpenAI client not configured")

    prompt = (
        "You are an HR assistant. You receive structured data about candidates' resumes. "
        "Each resume has a filename, a match_count (how many skills matched), "
        "and the matching_keywords.\n\n"
        f"Required skills: {', '.join(required_skills)}\n\n"
        f"Resume data (JSON):\n{json.dumps(scan_results, indent=2)}\n\n"
        "Task: Select the top candidates and explain briefly why. "
        "Return a short, clear summary in bullet points."
    )

    resp = await openai_client.responses.create(
        model="gpt-4.1-mini",
        input=prompt,
    )

    chunks = []
    for item in resp.output:
        if item.type == "message":
            for c in item.content:
                if c.type == "output_text":
                    chunks.append(c.text)
    return "\n".join(chunks).strip()


def analyze_with_gemini(required_skills: List[str], scan_results: List[Dict]) -> str:
    """
    Use Gemini to analyze PDF scan results (sync).
    """
    if not gemini_model:
        raise RuntimeError("Gemini is not configured")

    prompt = (
        "You are an HR assistant. You receive structured data about candidates' resumes. "
        "Each resume has a filename, a match_count (how many skills matched), "
        "and the matching_keywords.\n\n"
        f"Required skills: {', '.join(required_skills)}\n\n"
        f"Resume data (JSON):\n{json.dumps(scan_results, indent=2)}\n\n"
        "Task: Select the top candidates and explain briefly why. "
        "Return a short, clear summary in bullet points."
    )

    response = gemini_model.generate_content(prompt)
    return response.text.strip()


# ─────────────────────────────────────────────
# 6️⃣ STEP 4 – Orchestrator: JD → Keywords → Scan → Rank
# ─────────────────────────────────────────────
import asyncio

async def main():
    # 1) Paste job description (or load from file)
    job_description = """
    We are looking for a senior engineer with strong experience in Python, Swift,
    iOS development, Kubernetes, CI/CD, and cloud-native architectures.
    Experience with Docker, microservices, and REST APIs is a plus.
    """

    print("📝 Extracting keywords from job description…")
    required_skills = await extract_keywords_from_job_description(job_description, n_keywords=15)
    print("📌 Extracted keywords:", required_skills)

    # 2) Scan resumes
    print("🔎 Scanning resumes in folder:", RESUME_DIR)
    scan_results = scan_resumes_for_keywords(required_skills)
    print("✅ Scan complete. Found:", len(scan_results), "entries")

    # 3) Analyze with AI
    print("🤖 Analyzing candidates with AI…")

    # Try OpenAI first
    if openai_client:
        try:
            summary = await analyze_with_openai(required_skills, scan_results)
            print("\n✅ Final result (OpenAI):\n")
            print(summary)
            return
        except RateLimitError:
            print("⚠️ OpenAI quota exceeded. Falling back to Gemini…")
        except Exception as e:
            print(f"⚠️ OpenAI call failed ({e}). Trying Gemini if available…")

    # Fallback to Gemini
    if gemini_model:
        try:
            summary = analyze_with_gemini(required_skills, scan_results)
            print("\n✅ Final result (Gemini):\n")
            print(summary)
            return
        except Exception as e:
            print(f"❌ Gemini call failed: {e}")
    else:
        print("❌ Gemini key not configured; cannot fall back.")

    print("\n❌ All backends failed. Check keys, quotas, and network.")


# In a Jupyter notebook cell, just run:
await main()

📂 Using resume folder: /Users/JABERI/Documents/Dev/AI Agent/KodeKloud/AI Agent/AI-Agent-LAB/Resume Keyword Search Agent/resume
📝 Extracting keywords from job description…
⚠️ OpenAI keyword extraction failed (Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************OOYA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}}). Trying Gemini if available…
⚠️ Gemini keyword extraction failed: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
⚠️ All keyword extraction backends failed. Using naive split.
📌 Extracted keywo